In [5]:
from sentence_transformers import SentenceTransformer , util
import torch
import numpy as np
import pandas as pd
model = SentenceTransformer('paraphrase-MiniLM-L6-v2') # sentence transormer
import spacy
nlp = spacy.load('en_core_web_sm') # Spacy model that we use to partition sentences 


In [6]:
# Set device to gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [7]:
# Function to chunk sentences and generate embeddings
def process_text(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    # Move sentences to device
    chunk_embeddings = model.encode(sentences, convert_to_tensor=True, device=device) # move to gpu for quicker generation 
    story_embedding = torch.mean(chunk_embeddings, dim=0)
    return story_embedding.cpu()  # Move tensor back to CPU before returning | we need to do this in case we wanna use numpy arrays in the future

submissions = pd.read_json('output/filtered_and_balanced_submissions.json')
submissions['embedding'] = submissions['selftext'].apply(process_text)

submissions.to_pickle('output/paraphrase_mini_l6_embedded_averaged.pkl')
